<a href="https://colab.research.google.com/github/Conv-AI/Convai_Documentation/blob/main/Convai_Character_API_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convai Character API Tutorial

This interactive python notebook consists of code snippets that illustrates the different API endpoints related to creation and modification of characters along with interactions possible with the character.

In the notebook, we will carry out the following tasks:


1.   Create a character
2.   Modify the character
3.   Start a conversation with the character
4.   Delete the character


Note:
1. Before moving on with this tutorial, remember to get your API-Key from the profile section.
2. All characters are referenced by a **unique ID**, which is essential for all interactions with the character.

You can always refer to the documentation here for more details: [Link](https://docs.convai.com/api-docs/reference/api-reference/character-api)

### Imports

In [1]:
import requests
import json
import os
import base64
from IPython.display import Audio, display

### Set up API key

In [2]:
os.environ["CONVAI_API_KEY"]= "<your-api-key>"

### Creating a character

There are a few mandatory details we need to create a new character. Lets start by creating one of our own.

Say you are creating a new character that will act as a support NPC for the main character in a rpg game you are working on. Its decided that it will be a **male character**, named **Nova** with a simple background setup of an **adventurer from some kingdom**. Lets arrange out these information:
<br>

*   **Name:** Nova
*   **Backstory:** Nova was born in the Eastern Kingdom of Grabisco, the city of Rumbalara. His father, an adventurer, taught him everything he knew about combat, thievery, and adventuring. Nova had his father’s love of the arts, and he was particularly drawn to the magic arts. When he was old enough, he traveled with his father to the western lands of Grabisco. He stayed with the group of adventurers until they came to Rumbalara and then left to continue his adventures. Now he is traveling with his girlfriend, Epiphany, in search of his first band of adventurers.
*   **Voice:** Male

Note: For the list of avaialable voices, please refer to this list: [Voices](https://docs.convai.com/api-docs/reference/api-reference/text-to-speech-api#list-of-available-voices-and-their-supported-audio-file-encodings)

With the above details at hand we are ready to create a new character.

In [3]:
# Initializing a default character id for the new character to be created, to be used throught the notebook.
CHARACTER_ID = ""
CHARACTER_NAME = "Nova"

In [4]:
CREATE_CHARACTER_URL = "https://api.convai.com/character/create"

payload = json.dumps({
  "charName": "Nova",
  "voiceType": "MALE",
  "backstory": "Nova was born in the Eastern Kingdom of Grabisco, the city of Rumbalara. His father, an adventurer, taught him everything he knew about combat, thievery, and adventuring. Nova had his father’s love of the arts, and he was particularly drawn to the magic arts. When he was old enough, he traveled with his father to the western lands of Grabisco. He stayed with the group of adventurers until they came to Rumbalara and then left to continue his adventures. Now he is traveling with his girlfriend, Epiphany, in search of his first band of adventurers."
})
headers = {
  "CONVAI-API-KEY": os.getenv("CONVAI_API_KEY"),
  "Content-Type": "application/json"
}

response = requests.request("POST", CREATE_CHARACTER_URL, headers=headers, data=payload)

if response.status_code == 200:
  print("New character created")

  # A successful request will return a unique id for the character created.
  data = response.json()
  print("Character ID: ", data["charID"])
  CHARACTER_ID = data["charID"]

else:
  print("Failed to create new character")

  data = response.json()
  print("Error: ", data["ERROR"])

New character created


KeyError: ignored

### Modifying the character details

With the new character created, we might have to update some details of the character at some point. This can be achieved by a simle call to the corresponding endpoint and providing the necessary details.

Let's say, we need to update a small detail in the character backstory, which is to update the name of the character's girlfriend from **Epiphany** to **Shirley**. Thus the updated backstory should be:


*   **Backstory:** Nova was born in the Eastern Kingdom of Grabisco, the city of Rumbalara. His father, an adventurer, taught him everything he knew about combat, thievery, and adventuring. Nova had his father’s love of the arts, and he was particularly drawn to the magic arts. When he was old enough, he traveled with his father to the western lands of Grabisco. He stayed with the group of adventurers until they came to Rumbalara and then left to continue his adventures. Now he is traveling with his girlfriend, Shirley, in search of his first band of adventurers.

We execute the next set of code to update the details.

In [ ]:
MODIFY_CHARACTER_URL = "https://api.convai.com/character/update"

payload = json.dumps({
  "charID": "<character ID>",
  "backstory": "Nova was born in the Eastern Kingdom of Grabisco, the city of Rumbalara. His father, an adventurer, taught him everything he knew about combat, thievery, and adventuring. Nova had his father’s love of the arts, and he was particularly drawn to the magic arts. When he was old enough, he traveled with his father to the western lands of Grabisco. He stayed with the group of adventurers until they came to Rumbalara and then left to continue his adventures. Now he is traveling with his girlfriend, Shirley, in search of his first band of adventurers.",
})
headers = {
  "CONVAI-API-KEY": os.getenv("CONVAI_API_KEY"),
  "Content-Type": "application/json"
}

response = requests.request("POST", MODIFY_CHARACTER_URL, headers=headers, data=payload)

if response.status_code == 200:
  print("Character details updated")

  # A successful request will return SUCCESS as status.
  data = response.json()
  print("Update Status: ", data["STATUS"])

else:
  print("Failed to create new character")

  data = response.json()
  print("Error: ", data["ERROR"])


The above code can be used to update any detail of the character along with the character's voice, by providing the correpsonding character-id.

### Conversing with the character

TO start a conversation with the character, we need to understand a few things and provide the required details.

1.   Firstly, a character-id is essential to converse with a character. Kinda Obvious.
2.   Next, everytime we start conversing with a character, we create a **session**. A **session** is generally a group of a few back-and-forth conversation between the user and the character. They are referenced by the attribute called **sessionID**. This helps maintain context of conversation and extract relevant information from the chat-history belonging to the session. 
3.  To start a new session, we start by providing **sessionID** as **"-1"**, for the system to understand that a new set of conversation is starting. To continue an old conversation session, we pass the sessionID from that conversation. The system returns back a value for the sessionID, based on therequirement. If we are indicating the start of a new session, it returns a new **sessionID** otherwise it returns the old same value as passed in the request. Simple as that.
4.  The response from our API contains both the response and also the audio file of the respons from the character. To, opt out of receiving this audio data, we can set a flag attrbute called **voiceResponse** as **"false"**.
5.  A very crucial point to remember is that the audio data is returned as a base64 encoded data as a value of the **audio** key in the returned json data. The following code will show you how to handle the data.

While, the above instructions might look a little confusing, I suggest you follow along the code to get a better understanding.

In [ ]:
'''
We will try to execute a series of conversation with the character in the following lines.
Everytime we execute this block, we will create a new session for conversation. So we set "sessionID" as -1
'''
SESSION_ID = "-1"
CHARACTER_RESPONSE_URL = "https://api.convai.com/character/getResponse"
VOICE_REQUIRED_FLAG = "True"

while(True):
  userText = input("User: ") # The user enter whatever it wants to say to the character.

  '''
  Let's add a break condition.
  If the user enter "/stop" as input, we end the conversation streak.
  '''
  if userText == "/stop":
    break

  # Otherwise we continue with the required steps

  payload={
    'userText': userText,
    'charID': CHARACTER_ID,
    'sessionID': SESSION_ID,
    'voiceResponse': VOICE_REQUIRED_FLAG
  }

  headers = {
  "CONVAI-API-KEY": os.getenv("CONVAI_API_KEY"),
  }

  response = requests.request("POST", CHARACTER_RESPONSE_URL, headers=headers, data=payload)

  if response.status_code == 200:   # this indicates that the request has been processed succesfully
    data = response.json()  # Read the JSON data from the response

    # Display the reponse of the character to the user
    character_response = data["text"]
    print(CHARACTER_NAME+": "+character_response)

    '''
    Remember Points 2 & 3, where we dicussed a lot on "session" and "sessionID", here we are going to
    handle that.
    Since, we have sent "-1" as sessionID for the very first conversation, the system understanda that
    this is the start of a new conversation session. It returns a unique ID to mark this start. We
    have to reuse that to continue conversations by maintaining context of the session.
    If we send an old sessionID, the systems understands that the user is trying to maintain context
    from a particular conversation session and thus returns the same sessionID back as validation.
    '''
    SESSION_ID = data["sessionID"]

    '''
    Handling the audio data.
    As you can see, we have passed "True" for "voiceResponse" attribute. Thus we will get back some data
    in the "audio" attribute of the response. As mentioned in Point 5, the data is encoded in base64 format.
    So the natural course of action should be to decode the data. Then we write that data into an audio file
    to be played later.
    '''
    if VOICE_REQUIRED_FLAG:   # We move on with the next steps only if we are expecting some audio data
      decode_string = base64.b64decode(data["audio"])

      with open('audioResponse.wav','wb') as f:
        f.write(decode_string)
      
      display(Audio('/content/audioResponse.wav', autoplay=True))

  else:   # The status_code for the response is something other than 200.
    print("The conversation request has failed")
    
    data = response.json()  # Read the JSON data from the response
    print("ERROR: ", data["ERROR"])


That's it. You have successfully carried out conversation with your character. Do execute the above block again to converse again.

### Delete the character

You are done with your character, and you have cold-heartedly decided to dispose him. Quite a sad day for everyone. We can make a requst to the required endpoint to carry out this task.

Note: If you have followed the documentation, the **/delete** endpoint allows you to delete a character either by name or by the unique id. We prefer that you use the unique character-id to avoid any confusion. Sometimes, while using character-name, it will also delete other characters with the same name, whom you intended in keeping alive.

In [ ]:
DELETE_CHARACTER_URL = "https://api.convai.com/character/delete"

payload = json.dumps({
  "charID": CHARACTER_ID  # We have decided to kill our few minutes old character
})

headers = {
  "CONVAI-API-KEY": os.getenv("CONVAI_API_KEY"),
  "Content-Type": "application/json"
}

response = requests.request("POST", DELETE_CHARACTER_URL, headers=headers, data=payload)

if response.status_code == 200:
  print("Character deleted")

  # A successful request will return SUCCESS as status.
  data = response.json()
  print("Update Status: ", data["STATUS"])

else:
  print("Failed to delete the character")

  data = response.json()
  print("Error: ", data["ERROR"])

And that wraps up the basic tutorial on the Character endpoint provided by Convai. You can now go and create your own character and use them in the application that you are building.

In [ ]:
### 